In [1]:
import pandas as pd

In [2]:
data = pd.read_json('../data/mc.json')
data.head(5)

,address,condition_values,id,key,latitude,longitude,marker_index,name
0,北海道札幌市西区西町北２－１－６,"[1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1]",70893117,1003,43.077304,141.292534,0,西町店
1,北海道札幌市白石区南郷通２丁目南１０－１５,"[1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1]",70893118,1004,43.044633,141.400104,0,白石南郷通店
2,北海道札幌市中央区大通西１丁目地下街オーロラタウン,"[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1]",70893121,1007,43.061023,141.355693,0,オーロラタウン店
3,北海道札幌市中央区北五条西２０丁目１－１,"[1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]",70893122,1008,43.063588,141.325312,0,北５西２０店
4,北海道札幌市豊平区平岸二条１０丁目３－１２,"[1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1]",70893125,1011,43.030622,141.368216,0,平岸店


In [3]:
!pip list | grep mysql

mysqlclient          1.4.6      
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!pip install mysqlclient

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
# MySQL サーバーに接続
import MySQLdb

connection = MySQLdb.connect(
    host='127.0.0.1', user='root', port=13306, passwd='password', charset='utf8', database='geometry')
cursor = connection.cursor()

In [2]:
# テーブル作成
cursor.execute("""CREATE TABLE mc (
  `id` INT(10) UNSIGNED AUTO_INCREMENT PRIMARY KEY,
  `name` VARCHAR(64) NOT NULL,
  `address` VARCHAR(128) NOT NULL, 
  `geohash` VARCHAR(32) NOT NULL
) ENGINE = InnoDB;""")

0

In [41]:
cursor.execute("drop table mc;")

0

In [13]:
cursor.execute("ALTER TABLE mac ADD SPATIAL `location_index` (`location`); ")

0

In [38]:
# DB にデータを insert していく
# ('札幌駅', GeomFromText('POINT(141.350857 43.067656)')),]
rows = ["('{}', '{}', ST_GeoHash({}, {}, 12))".format(row['name'], row['address'], row['longitude'], row['latitude']) for index, row in data.iterrows()]
print(rows[0])

('西町店', '北海道札幌市西区西町北２－１－６', ST_GeoHash(141.292534094, 43.0773043276, 12))


In [39]:
# SQL の形
# INSERT INTO `test` (`name`, `location`) 
# VALUES ('札幌駅', GeomFromText('POINT(141.350857 43.067656)')),
#        ('上野駅', GeomFromText('POINT(139.776381 35.712297)')),
#        ('品川駅', GeomFromText('POINT(139.73876 35.628471)'));

insert = """
    INSERT INTO mc (name, address, geohash)
    VALUES {};
""".format(',\n\t'.join(rows))

print(insert)


    INSERT INTO mc (name, address, geohash)
    VALUES ('西町店', '北海道札幌市西区西町北２－１－６', ST_GeoHash(141.292534094, 43.0773043276, 12)),
	('白石南郷通店', '北海道札幌市白石区南郷通２丁目南１０－１５', ST_GeoHash(141.400104165, 43.0446331934, 12)),
	('オーロラタウン店', '北海道札幌市中央区大通西１丁目地下街オーロラタウン', ST_GeoHash(141.355693117, 43.0610230865, 12)),
	('北５西２０店', '北海道札幌市中央区北五条西２０丁目１－１', ST_GeoHash(141.325312165, 43.0635881741, 12)),
	('平岸店', '北海道札幌市豊平区平岸二条１０丁目３－１２', ST_GeoHash(141.36821602, 43.0306221912, 12)),
	('南新川店', '北海道札幌市北区北二十三条西１４丁目６－１', ST_GeoHash(141.327977125, 43.0876822741, 12)),
	('帯広柏林台店', '北海道帯広市西１８条南３－２７－３６', ST_GeoHash(143.159894664, 42.918490884, 12)),
	('札幌南二条店', '北海道札幌市中央区南二条西２丁目１', ST_GeoHash(141.35573607, 43.0580542823, 12)),
	('イオン釧路店', '北海道釧路郡釧路町桂木１－１－１', ST_GeoHash(144.405824423, 43.0091587556, 12)),
	('福住イトーヨーカドー店', '北海道札幌市豊平区福住二条１丁目', ST_GeoHash(141.403793918, 43.0201501787, 12)),
	('イオン千歳店', '北海道千歳市栄町６－５１', ST_GeoHash(141.64850902, 42.828842212, 12)),
	('イオン室蘭店', '北海道室蘭市東町２－４－３２', ST_GeoHash(141.02612207, 

In [40]:
cursor.execute(insert)

2898

In [33]:
cursor.close()